In [15]:
pip install seaborn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:
pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [19]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [20]:
import pandas as pd
import os
import zipfile

# over here i use get directory where notebook lays and than look for data folder in it
current_directory = os.getcwd()
data_directory = os.path.join(current_directory, 'data')

file_list = os.listdir(current_directory)

# than here we unzip all zipped files inside this dir if they are present
for file_name in file_list:
    if file_name.endswith('.zip'):
        file_path = os.path.join(current_directory, file_name)
        with zipfile.ZipFile(file_path, 'r') as zip_ref:
            zip_ref.extractall(data_directory)

data_frames = []
symbols = ['BTCUSDT', 'ETHUSDT', 'XRPUSDT', 'ADAUSDT', 'MATICUSDT']

# here we parse the data from our files and for symbol value we take part of a file name without stablecoin
for file_name in os.listdir(data_directory):
    if file_name.endswith('.csv'):
        for symbol in symbols:
            if file_name.startswith(symbol):
                file_path = os.path.join(data_directory, file_name)
                df = pd.read_csv(file_path, header=None)

                #symbol extraction
                symbol_name = file_name.split('USDT')[0]
                
                df['Symbol'] = symbol_name
                data_frames.append(df)
                break 

# concat all extracted data to one dataframe
combined_data = pd.concat(data_frames, ignore_index=True)

# our column names
column_names = [
    'Kline Open time in unix time format',
    'Open Price',
    'High Price',
    'Low Price',
    'Close Price',
    'Volume',
    'Kline Close time in unix time format',
    'Quote Asset Volume',
    'Number of Trades',
    'Taker buy base asset volume during this period',
    'Taker buy quote asset volume during this period',
    'ignore',
    'Symbol'
]

# set names for columns in dataframe
combined_data.columns = column_names

# check for null values (data corruption validation)
null_values = combined_data.isnull().sum()
print(null_values)

# here we want to check whether there are 2x variance open prices in dataset
variance_per_symbol = combined_data.groupby('Symbol')['Open Price'].var()
count_large_values_2x_variance = (combined_data.groupby('Symbol')['Open Price']
                                  .apply(lambda x: (x > (2 * variance_per_symbol[x.name])).sum()))

print("\n\nSymbols with count of 'Open Price' values larger than 2 times variance:")
print(count_large_values_2x_variance)

# Calculate total price move in value for each symbol
total_price_move = combined_data.groupby('Symbol')['Close Price'].last() - combined_data.groupby('Symbol')['Open Price'].first()
first_open_prices = combined_data.groupby('Symbol')['Open Price'].first()
relative_percent_change = ((combined_data.groupby('Symbol')['Close Price'].last() - first_open_prices) / first_open_prices) * 100

# Displaying results
print("\n\nTotal price move in value for each symbol:")
print(total_price_move)
print("\n\nRelative percentage change from first open price for each symbol:")
print(relative_percent_change)


# and then our fully composed file with all data
combined_data.to_csv(os.path.join(current_directory, 'combined_file_with_symbol.csv'), index=False)


Kline Open time in unix time format                0
Open Price                                         0
High Price                                         0
Low Price                                          0
Close Price                                        0
Volume                                             0
Kline Close time in unix time format               0
Quote Asset Volume                                 0
Number of Trades                                   0
Taker buy base asset volume during this period     0
Taker buy quote asset volume during this period    0
ignore                                             0
Symbol                                             0
dtype: int64


Symbols with count of 'Open Price' values larger than 2 times variance:
Symbol
ADA      1176
BTC         0
ETH         0
MATIC    1176
XRP      1176
Name: Open Price, dtype: int64


Total price move in value for each symbol:
Symbol
ADA         0.2832
BTC      7636.2100
ETH       363.2400
MATIC  

In [21]:
# and that is cleaning after ourselves, used to delete all zip files in directory after extraction from them

import os

current_directory = os.getcwd()
data_directory = os.path.join(current_directory, 'data')

if os.path.exists(data_directory):
    file_list = os.listdir(data_directory)

    for file_name in file_list:
        if file_name.endswith('.zip'):
            file_path = os.path.join(data_directory, file_name)
            os.remove(file_path)

In [27]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

# same manipulations here, parsing data to dataframe
df = pd.read_csv('combined_file_with_symbol.csv')

# convertion to readable form
df['Open_Time'] = pd.to_datetime(df['Kline Open time in unix time format'], unit='ms')
df['Close_Time'] = pd.to_datetime(df['Kline Close time in unix time format'], unit='ms')

# additional copy of close price to work on with
df['Close_Price'] = df['Close Price']

# our desired plot (Price + Liquidity)
def plot_symbol_data(symbol):
    symbol_data = df[df['Symbol'] == symbol]
    
    plt.figure(figsize=(12, 6))
    plt.plot(symbol_data['Close_Time'], symbol_data['Close_Price'])
    plt.xlabel('Time')
    plt.ylabel('Price')
    plt.title(f'Price of {symbol}')
    plt.xticks(rotation=45)
    
    plt.tight_layout()
    plt.show()

    plt.figure(figsize=(12, 6))
    plt.plot(symbol_data['Close_Time'], symbol_data['Volume'])
    plt.xlabel('Time')
    plt.ylabel('Volume')
    plt.title(f'Liquidity of {symbol}')
    plt.xticks(rotation=45)
    
    plt.tight_layout()
    plt.show()

# dropdown widget for symbol selection
symbols = df['Symbol'].unique()
symbol_dropdown = widgets.Dropdown(options=symbols, description='Symbol:')

# making plot interactive
interactive_plot = widgets.interactive(plot_symbol_data, symbol=symbol_dropdown)

# plot display with widget
display(widgets.VBox([interactive_plot]))


## The data shown above can inform observer about how well given asset can absorb money, the more liquidity you see and the least connected price range to it is a direct proof that the asset is liquid and in reverse - the least liqudity and wider price moves - proof of low liquidity

In [23]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

# same manipulations here, parsing data to dataframe
df = pd.read_csv('combined_file_with_symbol.csv')

# convertion to readable form
df['Open_Time'] = pd.to_datetime(df['Kline Open time in unix time format'], unit='ms')
df['Close_Time'] = pd.to_datetime(df['Kline Close time in unix time format'], unit='ms')

# additional copy of close price to work on with
df['Close_Price'] = df['Close Price']

# here we calculate price difference in given timeframe
def calculate_price_difference(symbol, timeframe):
    timeframe_ms = timeframe * 60 * 60 * 1000  # hour to millisecond
    
    # data grouping based on given symbol and given timeframe
    grouped = df[df['Symbol'] == symbol].groupby(pd.Grouper(key='Open_Time', freq=f'{timeframe}H'))
    
    # and lastly calculation of price difference in group
    price_diff = grouped.apply(lambda x: x['Close Price'].iloc[-1] - x['Open Price'].iloc[0])
    
    return price_diff

# here we calculate volume for given timeframe
def calculate_volume(symbol, timeframe):
    timeframe_ms = timeframe * 60 * 60 * 1000  # hour to millisecond
    
    # same grouping as done before
    grouped = df[df['Symbol'] == symbol].groupby(pd.Grouper(key='Open_Time', freq=f'{timeframe}H'))
    
    # volume calculation for group
    volume = grouped['Volume'].sum()
    
    return volume

# here we calculate total trades traded in given timeframe
def calculate_number_of_trades(symbol, timeframe):
    timeframe_ms = timeframe * 60 * 60 * 1000  # hour to millisecond
    
    # same grouping as done before
    grouped = df[df['Symbol'] == symbol].groupby(pd.Grouper(key='Open_Time', freq=f'{timeframe}H'))
    
    # total trades in given group calculation
    num_trades = grouped['Number of Trades'].sum()
    
    return num_trades

# here is slightly different part, its correlation between price difference, volume and number of trades
def calculate_correlation(symbol, timeframe):
    price_diff = calculate_price_difference(symbol, timeframe)
    volume = calculate_volume(symbol, timeframe)
    num_trades = calculate_number_of_trades(symbol, timeframe)
    
    # smalled combined dataframe to work with
    combined_df = pd.DataFrame({'Price Difference': price_diff, 'Volume': volume, 'Number of Trades': num_trades})
    
    # and correlation matrix calculation for it
    correlation_matrix = combined_df.corr()
    
    return correlation_matrix

# here we render our plots, pretty straightforward and simple
def update_plot(symbol, timeframe):
    price_diff = calculate_price_difference(symbol, timeframe)
    volume = calculate_volume(symbol, timeframe)
    num_trades = calculate_number_of_trades(symbol, timeframe)
    correlation = calculate_correlation(symbol, timeframe)
    
    plt.figure(figsize=(15, 12))
    
    # plot for price difference
    plt.subplot(2, 2, 1)
    plt.plot(price_diff.index, price_diff.values)
    plt.xlabel('Timeframe')
    plt.ylabel('Price Difference')
    plt.title(f'Price Difference for {symbol} over {timeframe} hours timeframe')
    plt.xticks(rotation=45)
    
    # plot for volume
    plt.subplot(2, 2, 2)
    plt.bar(volume.index, volume.values, color='orange')
    plt.xlabel('Timeframe')
    plt.ylabel('Volume')
    plt.title(f'Volume for {symbol} over {timeframe} hours timeframe')
    plt.xticks(rotation=45)
    
    # plot for number of trades
    plt.subplot(2, 2, 3)
    plt.plot(num_trades.index, num_trades.values, color='green')
    plt.xlabel('Timeframe')
    plt.ylabel('Number of Trades')
    plt.title(f'Number of Trades for {symbol} over {timeframe} hours timeframe')
    plt.xticks(rotation=45)
    
    # plot for correlation matrix
    plt.subplot(2, 2, 4)
    plt.title(f'Correlation Matrix for {symbol} over {timeframe} hours timeframe')
    plt.imshow(correlation, cmap='coolwarm', interpolation='nearest')
    plt.colorbar()
    plt.xticks(range(len(correlation.columns)), correlation.columns, rotation=45)
    plt.yticks(range(len(correlation.columns)), correlation.columns)
    
    plt.tight_layout()
    plt.show()

# symbols in dataframe
symbols = df['Symbol'].unique()

# dropdowns for symbol and timeframe group
symbol_dropdown = widgets.Dropdown(options=symbols, description='Symbol:')
timeframe_dropdown = widgets.Dropdown(options=[3, 6, 12], description='Timeframe (hours):')

# interactive plot as done before
interactive_plot = widgets.interactive(update_plot, symbol=symbol_dropdown, timeframe=timeframe_dropdown)

# just simple grouping of widgets to one box
input_widgets = widgets.VBox([symbol_dropdown, timeframe_dropdown])
output_widget = widgets.interactive_output(update_plot, {'symbol': symbol_dropdown, 'timeframe': timeframe_dropdown})

# display all
display(input_widgets, output_widget)


Output()

## Charts displayed above are a bit more complex, but more informative in practical aspects. Lets suppose observer wants to build an automated trading solution that will work on fixed timeframes and do simple operation: buy at the beginning of an interval and sell at the end. Shown charts proof that this type of approach worked for our dataset, because for most cases price difference between sell and buy is positive for the whole dataset, which means that resulting captured price movement is profitable. Than as interesting insights you can see correlation between price difference, volume traded in each interval and number of trades happened

In [24]:
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import ipywidgets as widgets
from IPython.display import display
%matplotlib widget

# same manipulations here, parsing data to dataframe
df = pd.read_csv('combined_file_with_symbol.csv')

# convertion to readable form
df['Open_Time'] = pd.to_datetime(df['Kline Open time in unix time format'], unit='ms')
df['Close_Time'] = pd.to_datetime(df['Kline Close time in unix time format'], unit='ms')

# additional copy of close price to work on with
df['Close_Price'] = df['Close Price']

# all the symbols in given dataframe
symbols = df['Symbol'].unique()

# here we will make an interesting 3d interactive plot to show some insights
def plot_3d_surface(symbol, timeframe):
    # data preparation for plotting, work only with matching symbol, index as open time and resampling for given timeframe group + dropping whats missing after resampling (to reduce possible missing values)
    filtered_data = df[df['Symbol'] == symbol].set_index('Open_Time').resample(f'{timeframe}H').last().dropna()

    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')

    # here we take whats will be rendered on plot, prices, proportion of market buy and sell volume in given group and number of trades made
    prices = filtered_data['Close_Price'].values
    proportion = filtered_data['Taker buy base asset volume during this period'] / filtered_data['Taker buy quote asset volume during this period']
    num_trades = filtered_data['Number of Trades'].values

    # meshgrid for trades and prices
    num_trades_grid, prices_grid = np.meshgrid(num_trades, prices)

    # proportion reshape (needed for proper 3d visualization)
    proportion_grid = np.tile(proportion.values, (len(prices), 1))

    ax.plot_surface(num_trades_grid, prices_grid, proportion_grid, cmap='viridis')
    ax.set_xlabel('Number of Trades')
    ax.set_ylabel('Price')
    ax.set_zlabel('Proportion buys / sells')
    ax.set_title(f'Price vs Number of Trades for {symbol} ({timeframe}H timeframe)')

    plt.show()

# dropdowns for our interactive plot
symbol_dropdown = widgets.Dropdown(options=symbols, description='Symbol:')
timeframe_dropdown = widgets.Dropdown(options=[3, 6, 12], description='Timeframe (hours):')

# the plot itself
interactive_plot_3d = widgets.interactive(plot_3d_surface, symbol=symbol_dropdown, timeframe=timeframe_dropdown)

# display all
display(widgets.VBox([interactive_plot_3d]))

## This 3D plot you saw contains interesting insights on how number of trades, price and ratio between sells and buys are connected with each other in our dataset. In some assets gaps in plot can signal about low liquidity for both sides of a market and as a result be a signal of fast price change. Practical usage of this is to determine this gap, compare it to gaps recorded and if its more than average - probably reasonable to react for it and look for some additional evidence about in which side market will move

In [25]:
import pandas as pd
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display

# same manipulations here, parsing data to dataframe
df = pd.read_csv('combined_file_with_symbol.csv')

# scatter matrix plotting, a bit strange thing on first sight
def plot_scatter_matrix(symbol, sample_size):
    subset = df[df['Symbol'] == symbol].sample(n=sample_size, random_state=42) # random sample for given size
    sns.pairplot(subset[['Volume', 'Close Price', 'Number of Trades']])
    plt.suptitle(f"Scatter Matrix for {symbol}")
    plt.show()

# dropdowns 
symbols = df['Symbol'].unique()
symbol_dropdown = widgets.Dropdown(options=symbols, description='Symbol:')
sample_size_slider = widgets.IntSlider(value=100, min=50, max=1000, step=50, description='Sample Size:')

# interactive plot creation
interactive_scatter_matrix = widgets.interactive(plot_scatter_matrix, symbol=symbol_dropdown, sample_size=sample_size_slider)

# display all
display(widgets.VBox([interactive_scatter_matrix]))


In [26]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

# same manipulations here, parsing data to dataframe
df = pd.read_csv('combined_file_with_symbol.csv')

# convertion to readable form
df['Kline Close time in unix time format'] = pd.to_datetime(df['Kline Close time in unix time format'], unit='ms')

def plot_candlestick_with_ma(symbol, window_size):
    subset = df[df['Symbol'] == symbol]

    fig, ax = plt.subplots(figsize=(10, 6))
    ax.plot(subset['Kline Close time in unix time format'], subset['Close Price'], label='Close Price')

    # our moving averages, extremely simple and lagging :) indicator that is used in asset price movement analysis
    ma_short = subset['Close Price'].rolling(window=window_size).mean()
    ma_long = subset['Close Price'].rolling(window=window_size * 3).mean()

    ax.plot(subset['Kline Close time in unix time format'], ma_short, label=f'MA ({window_size} periods)')
    ax.plot(subset['Kline Close time in unix time format'], ma_long, label=f'MA ({window_size * 3} periods)')
    ax.legend()
    ax.set_title(f'Price chart for {symbol} with Moving Averages')
    ax.set_xlabel('Time')
    ax.set_ylabel('Price')
    plt.xticks(rotation=45)
    plt.show()

# dropdowns to change data
symbols = df['Symbol'].unique()
symbol_dropdown = widgets.Dropdown(options=symbols, description='Symbol:')
window_size_slider = widgets.IntSlider(value=10, min=5, max=50, step=5, description='Window Size:')

# interactive plot itself
interactive_candlestick_ma = widgets.interactive(plot_candlestick_with_ma, symbol=symbol_dropdown, window_size=window_size_slider)

# display all
display(widgets.VBox([interactive_candlestick_ma]))


## What is shown above is an informative but not very precise indicator and visualization from which you can see, try to find and analyse when price movement is expected to change its direction from its local trend - it is when smaller and larger moving averages intersect/or touch each other